In [1]:
from laboneq.simple import SHFQC
from laboneq.dsl.device import DeviceSetup, create_connection
import numpy as np
from copy import deepcopy

host = "192.168.0.83"
#instantiate ad device setup object
setup = DeviceSetup(uid="kaist_setup")
#add a dataserver
setup.add_dataserver(host=host, port="8004",uid="kaist_server")

setup.add_instruments(SHFQC(uid="kaist_shfqc",server_uid="kaist_server", address="DEV12256", device_options="SHFQC/PLUS/QC6CH"))

setup.instruments

[SHFQC(
│   uid='kaist_shfqc',
│   interface='1GbE',
│   connections=[],
│   server_uid='kaist_server',
│   address='DEV12256',
│   device_options='SHFQC/PLUS/QC6CH',
│   reference_clock_source=None,
│   device_class=0
)
]

In [2]:
for ch_idx, q_idx in enumerate([1]):# enumerate 안에 len(list) 강제하고싶음
    #QA
    setup.add_connections(
        "kaist_shfqc",
        create_connection(to_signal=f"q{q_idx}/measure", ports="QACHANNELS/0/OUTPUT")) #signal_type="iq"
    setup.add_connections(
        "kaist_shfqc",
        create_connection(to_signal=f"q{q_idx}/acquire", ports="QACHANNELS/0/INPUT" )) #signal_type="acquire"
    #SG
    setup.add_connections(
        "kaist_shfqc",
        create_connection(to_signal=f"q{q_idx}/drive", ports=f"SGCHANNELS/{ch_idx}/OUTPUT")) #signal_type="iq" only iq allowed for SG??
    setup.add_connections(
        "kaist_shfqc",
        create_connection(to_signal=f"q{q_idx}/drive_ef", ports=f"SGCHANNELS/{ch_idx}/OUTPUT")) #signal_type="iq" only iq allowed for SG??
    setup.add_connections(
        "kaist_shfqc",
        create_connection(to_signal=f"q{q_idx}/drive_cr", ports=f"SGCHANNELS/{ch_idx}/OUTPUT"))#signal_type="iq"


In [3]:
from qpu_types.transmon import TransmonQubit, TransmonQubitParameters
from helper import load_qubit_parameters, save_qubit_parameters
from operations import TransmonOperations
from laboneq.dsl.quantum.qpu import QPU, QuantumPlatform



#_loaded_qubits = load_qubit_parameters(filename="latest") 
qubits = TransmonQubit.from_device_setup(setup) # Also can be done with load_qubit_parameters 
#qubits = list(_loaded_qubits.values())


qpu = QPU(quantum_elements=qubits, quantum_operations=TransmonOperations())
#quantum_platform = QuantumPlatform(setup=setup, qpu=qpu)

# qpu.topology.add_edge("bus", "q16", "q17")
# qpu.topology.add_edge("bus", "q17", "q18")
# qpu.topology.add_edge("bus", "q18", "q19")
# qpu.topology.plot()




from laboneq.simple import workflow
folder_store = workflow.logbook.FolderStore("./experiment_store") 
folder_store.activate()
#folder_store.deactivate()
workflow.logbook.LoggingStore().activate()
#workflow.logbook.LogbookStore().deactivate()


In [4]:
from laboneq.simple import Session
session = Session(setup)
session.connect(ignore_version_mismatch=True)
#session.disconnect()

[2025.09.02 19:10:37.556] INFO    Logging initialized from [Default inline config in laboneq.laboneq_logging] logdir is /Users/yalgaeahn/JSAHN/qubit-experiment/laboneq_output/log
[2025.09.02 19:10:37.558] INFO    VERSION: laboneq 2.58.0
[2025.09.02 19:10:37.559] INFO    Connecting to data server at 192.168.0.83:8004
[2025.09.02 19:10:37.599] INFO    Connected to Zurich Instruments LabOne Data Server version 25.07.0.507 at 192.168.0.83:8004
[2025.09.02 19:10:37.647] INFO    Configuring the device setup
[2025.09.02 19:10:37.677] INFO    The device setup is configured


In [43]:
qubits[0].parameters.readout_range_out = -5
qubits[0].parameters.readout_amplitude = 0.4

In [44]:
from laboneq_applications.experiments import resonator_spectroscopy
q = qubits[0]

###############################################################
frequencies = q.parameters.readout_resonator_frequency + np.linspace(-2e6,+2e6,201)
################################################################
options = resonator_spectroscopy.experiment_workflow.options()
options.count(1024*2)
options.update(True)
#options.fit_complex_resonator(True)

#print(workflow.show_fields(options))

###################################################################
res_spec = resonator_spectroscopy.experiment_workflow(
    session=session,
    qpu=qpu,
    qubit=q,
    frequencies=frequencies,
    options=options,
)

res_spec_result = res_spec.run()
#print(res_spec_result.tasks['analysis_workflow'].output)

[2025.09.02 19:32:55.699] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.02 19:32:55.700] INFO      Workflow 'resonator_spectroscopy': execution started at 2025-09-02            
[2025.09.02 19:32:55.700] INFO      10:32:55.699548Z                                                              
[2025.09.02 19:32:55.701] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.02 19:32:55.716] INFO    Task 'temporary_qpu': started at 2025-09-02 10:32:55.714849Z
[2025.09.02 19:32:55.717] INFO    Task 'temporary_qpu': ended at 2025-09-02 10:32:55.717506Z
[2025.09.02 19:32:55.719] INFO    Task 'temporary_quantum_elements_from_qpu': started at 2025-09-02 
[2025.09.02 19:32:55.719] INFO    10:32:55.719138Z
[2025.09.02 19:32:55.720] INFO    Task 'temporary_quantum_elements_from_qpu': ended at 2025-09-02 10:32:55.720079Z
[2025.09.02 19:32:55.721] INFO    Task 'create_experiment': started at 2025-09-02 10:

In [42]:
from laboneq_applications.experiments import resonator_spectroscopy_amplitude

q = qubits[0]

###############################################################
frequencies = q.parameters.readout_resonator_frequency + np.linspace(-2e6,+2e6,101)
################################################################
options = resonator_spectroscopy_amplitude.experiment_workflow.options()
options.count(1024*2)
options.update(True)
#options.fit_complex_resonator(True)

#print(workflow.show_fields(options))

###################################################################
res_spec_amp = resonator_spectroscopy_amplitude.experiment_workflow(
    session=session,
    qpu=qpu,
    qubit=q,
    frequencies=frequencies,
    amplitudes=np.linspace(0,1.0,21),
    options=options,
)

res_spec_amp_result = res_spec_amp.run()
#print(res_spec_result.tasks['analysis_workflow'].output)

[2025.09.02 19:31:56.651] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.02 19:31:56.652] INFO      Workflow 'resonator_spectroscopy_amplitude': execution started at 2025-09-02  
[2025.09.02 19:31:56.652] INFO      10:31:56.651078Z                                                              
[2025.09.02 19:31:56.653] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.02 19:31:56.669] INFO    Task 'temporary_qpu': started at 2025-09-02 10:31:56.669692Z
[2025.09.02 19:31:56.671] INFO    Task 'temporary_qpu': ended at 2025-09-02 10:31:56.671022Z
[2025.09.02 19:31:56.673] INFO    Task 'temporary_quantum_elements_from_qpu': started at 2025-09-02 
[2025.09.02 19:31:56.673] INFO    10:31:56.672756Z
[2025.09.02 19:31:56.674] INFO    Task 'temporary_quantum_elements_from_qpu': ended at 2025-09-02 10:31:56.673925Z
[2025.09.02 19:31:56.675] INFO    Task 'create_experiment': started at 2025-09-02 10:

In [ ]:
qubits[0].parameters.readout_resonator_frequency

In [ ]:
qubits[0].readout_parameters()

In [ ]:
import matplotlib.pyplot as plt
raw_data = res_spec_result.output['q1/result'].data
plt.plot(raw_data.imag)

In [ ]:
res_spec_result.tasks['analysis_workflow'].tasks['plot_real_imaginary'].output

In [ ]:
res_spec_result.tasks['analysis_workflow'].tasks['plot_magnitude_phase'].output

In [ ]:
res_spec_result.tasks['analysis_workflow'].tasks['plot_raw_complex_data_1d'].output['q1']

In [45]:
qubits[0].parameters.spectroscopy_amplitude =0.5
qubits[0].parameters.spectroscopy_length
qubits[0].parameters.resonance_frequency_ge = 4.2846e9
qubits[0].parameters.drive_range = 10
qubits[0].parameters.readout_range_in = 10
qubits[0].parameters.readout_range_out = -5
qubits[0].parameters.readout_amplitude = 0.4
qubits[0].parameters.ge_drive_length =64e-9
qubits[0].parameters.drive_lo_frequency = 4.2e9
qubits[0].parameters.readout_lo_frequency = 6.8e9
qubits[0].parameters.readout_resonator_frequency = 6.7275e9

In [38]:
qubits[0].readout_parameters()

('measure',
 {'amplitude': 0.4,
  'length': 2e-06,
  'pulse': {'function': 'GaussianSquare',
   'sigma': 0.2,
   'risefall_sigma_ratio': 3.0}})

In [13]:
qubits[0].spectroscopy_parameters()

('drive',
 {'amplitude': 1.0,
  'length': 5e-06,
  'pulse': {'function': 'GaussianSquare',
   'sigma': 0.2,
   'risefall_sigma_ratio': 3.0}})

In [59]:
qubits[0].parameters.drive_range

10

# Qubit Spectroscopy

In [56]:
from copy import deepcopy
from laboneq_applications.experiments import qubit_spectroscopy
q = qubits[0]
###############################################################

frequencies = 4.2846e9 +np.linspace(-5e6,5e6,201)
################################################################
options = qubit_spectroscopy.experiment_workflow.options()
options.update(False)
options.count(1024*4)


#print(workflow.show_fields(options))

###################################################################
qubit_spec = qubit_spectroscopy.experiment_workflow(
    session=session,
    qpu=qpu,
    qubits=q,
    frequencies=frequencies,
    options=options
)

qubit_spec_result = qubit_spec.run()
#qubit_spec_compiled = session.compile(qubit_spectroscopy.create_experiment(qpu=qpu,qubit=q,frequencies=frequencies, options=options))
qubit_parameters = qubit_spec_result.tasks['analysis_workflow'].output
print(qubit_parameters)

[2025.09.02 19:38:21.378] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.02 19:38:21.378] INFO      Workflow 'qubit_spectroscopy': execution started at 2025-09-02                
[2025.09.02 19:38:21.379] INFO      10:38:21.377731Z                                                              
[2025.09.02 19:38:21.379] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.02 19:38:21.391] INFO    Task 'temporary_qpu': started at 2025-09-02 10:38:21.391500Z
[2025.09.02 19:38:21.393] INFO    Task 'temporary_qpu': ended at 2025-09-02 10:38:21.392927Z
[2025.09.02 19:38:21.394] INFO    Task 'temporary_quantum_elements_from_qpu': started at 2025-09-02 
[2025.09.02 19:38:21.394] INFO    10:38:21.394453Z
[2025.09.02 19:38:21.395] INFO    Task 'temporary_quantum_elements_from_qpu': ended at 2025-09-02 10:38:21.395839Z
[2025.09.02 19:38:21.397] INFO    Task 'create_experiment': started at 2025-09-02 10:

In [55]:
qubits[0].parameters.drive_range = 10
qubits[0].parameters.spectroscopy_amplitude =1.0

In [51]:
from copy import deepcopy
from laboneq_applications.experiments import qubit_spectroscopy_amplitude
q = qubits[0]
###############################################################
frequencies = q.parameters.resonance_frequency_ge +np.linspace(-5e6,5e6,101)
################################################################
options = qubit_spectroscopy_amplitude.experiment_workflow.options()
options.update(False)



#print(workflow.show_fields(options))

###################################################################
qubit_spec_amp = qubit_spectroscopy_amplitude.experiment_workflow(
    session=session,
    qpu=qpu,
    qubits=q,
    frequencies=frequencies,
    amplitudes=np.linspace(0,1.0,21),
    options=options
)

qubit_spec_amp_result = qubit_spec_amp.run()
#qubit_spec_compiled = session.compile(qubit_spectroscopy.create_experiment(qpu=qpu,qubit=q,frequencies=frequencies, options=options))
qubit_parameters = qubit_spec_result.tasks['analysis_workflow'].output
print(qubit_parameters)

[2025.09.02 19:37:01.343] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.02 19:37:01.343] INFO      Workflow 'qubit_spectroscopy_amplitude': execution started at 2025-09-02      
[2025.09.02 19:37:01.344] INFO      10:37:01.342690Z                                                              
[2025.09.02 19:37:01.344] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.02 19:37:01.358] INFO    Task 'temporary_qpu': started at 2025-09-02 10:37:01.358678Z
[2025.09.02 19:37:01.360] INFO    Task 'temporary_qpu': ended at 2025-09-02 10:37:01.360062Z
[2025.09.02 19:37:01.362] INFO    Task 'temporary_quantum_elements_from_qpu': started at 2025-09-02 
[2025.09.02 19:37:01.362] INFO    10:37:01.362141Z
[2025.09.02 19:37:01.363] INFO    Task 'temporary_quantum_elements_from_qpu': ended at 2025-09-02 10:37:01.363399Z
[2025.09.02 19:37:01.365] INFO    Task 'create_experiment': started at 2025-09-02 10:

In [ ]:
qubits[0].parameters.ge_drive_length =64e9

# Rabi amplitude

In [ ]:
from laboneq_applications.experiments import amplitude_rabi
q = qubits[0]

#######################################################################
amplitudes = np.linspace(0,1.0,21)
#######################################################################
options = amplitude_rabi.experiment_workflow.options()
options.update(False)
options.count(1024*2)
options.use_cal_traces(True)
#print(workflow.show_fields(options))

###################################################################
rabi_amp = amplitude_rabi.experiment_workflow(
    session=session,
    qpu=qpu,
    qubits=q,
    amplitudes=amplitudes,
    #options=options
)

rabi_amp_result = rabi_amp.run()
#qubit_spec_compiled = session.compile(amplitude_rabi.create_experiment(qpu=qpu,qubit=q,amplitudes=amplitudes, options=options))
#print(rabi_amp_result.tasks['analysis_workflow'].output)

# Ramsey

In [ ]:
from laboneq_applications.experiments import ramsey
q = qubits[0]
temporary_parameters = q.parameters.copy()
#######################################################################
delays = np.linspace(0,1e-6,10)
detunings = 0.2e6
#######################################################################
# options = ramsey.experiment_workflow.options()
# options.transition("ge")
# options.cal_states("ge")
# options.update(True)
#print(workflow.show_fields(options))

###################################################################
ramsey_freq = ramsey.experiment_workflow(
    session=session,
    qpu=qpu,
    qubits=q,
    delays=delays,
    detunings=detunings,
    #temporary_parameters={q.uid : temporary_parameters},
    #options=options
)

ramsey_freq_result = ramsey_freq.run()
#ramsey_compiled = session.compile(ramsey.create_experiment(qpu=qpu,qubit=q,delays=delays,detunings=detunings, options=options))
print(ramsey_freq_result.tasks['analysis_workflow'].output)

# Error Amplification

In [ ]:
from laboneq_applications.experiments import amplitude_fine
q = qubits[3]
temporary_parameters = q.parameters.copy()

#######################################################################
repetitions =np.arange(1,50)
#######################################################################
options = amplitude_fine.experiment_workflow_x180.options()
options.update(True)
options.use_cal_traces(True)
#print(workflow.show_fields(options))

###################################################################
error_amp = amplitude_fine.experiment_workflow_x180(
    session=session,
    qpu=qpu,
    qubits=q,
    repetitions=repetitions,
    temporary_parameters={q.uid : temporary_parameters},
    options=options
)

error_amp_result = error_amp.run()
#qubit_spec_compiled = session.compile(amplitude_rabi.create_experiment(qpu=qpu,qubit=q,amplitudes=amplitudes, options=options))
print(error_amp_result.tasks['analysis_workflow'].output)

In [ ]:
from laboneq_applications.experiments import amplitude_fine
q = qubits[3]
temporary_parameters = q.parameters.copy()

#######################################################################
repetitions =np.arange(1,50)
#######################################################################
options = amplitude_fine.experiment_workflow_x90.options()
options.update(True)
options.use_cal_traces(True)
#print(workflow.show_fields(options))

###################################################################
error_amp_half = amplitude_fine.experiment_workflow_x90(
    session=session,
    qpu=qpu,
    qubits=q,
    repetitions=repetitions,
    temporary_parameters={q.uid : temporary_parameters},
    options=options
)

error_amp_half_result = error_amp_half.run()
#qubit_spec_compiled = session.compile(amplitude_rabi.create_experiment(qpu=qpu,qubit=q,amplitudes=amplitudes, options=options))
print(error_amp_half_result.tasks['analysis_workflow'].output)

# Dispersive Shift

In [ ]:
from laboneq_applications.experiments import dispersive_shift
q = qubits[3]
temporary_parameters = q.parameters.copy()

#######################################################################
frequencies=q.parameters.readout_resonator_frequency + np.linspace(-5e6,+5e6,101)
states=["g","e"]

#######################################################################
options = dispersive_shift.experiment_workflow.options()
options.update(True)
#options.use_cal_traces(True)
#print(workflow.show_fields(options))

###################################################################
dispersive = dispersive_shift.experiment_workflow(
    session=session,
    qpu=qpu,
    qubit=q,
    frequencies=frequencies,
    states=states,
    temporary_parameters={q.uid : temporary_parameters},
    options=options
)

dispersive_result = dispersive.run()
#qubit_spec_compiled = session.compile(amplitude_rabi.create_experiment(qpu=qpu,qubit=q,amplitudes=amplitudes, options=options))
print(dispersive_result.tasks['analysis_workflow'].output)

# IQ blob

In [ ]:
from laboneq_applications.experiments import iq_blobs
q = qubits[2]
#temporary_parameters = q.parameters.copy()

#######################################################################

states=["g","e"]

#######################################################################
options = iq_blobs.experiment_workflow.options()
#options.update(False)
#options.use_cal_traces(True)
#print(workflow.show_fields(options))

###################################################################
iq = iq_blobs.experiment_workflow(
    session=session,
    qpu=qpu,
    qubits=q,
    states=states,
    temporary_parameters={q.uid : temporary_parameters},
    options=options
)

iq_result = iq.run()
#qubit_spec_compiled = session.compile(amplitude_rabi.create_experiment(qpu=qpu,qubit=q,amplitudes=amplitudes, options=options))


# T1

In [ ]:
from laboneq_applications.experiments import lifetime_measurement

options = lifetime_measurement.experiment_workflow.options()
q=qubits[0]
t1 = lifetime_measurement.experiment_workflow(
    session=session,
    qpu=qpu,
    qubits=q,
    delays=np.linspace(0,40e-6,201)
    )

# Save Qubit Parameters

In [ ]:

from helper import load_qubit_parameters, save_qubit_parameters
save_qubit_parameters(qubits={ qubit.uid : qubit for qubit in qubits}, save_folder="./qubit_parameters", filename="kaist_mj")